In [1]:
import pandas as pd
from datetime import datetime
import googlemaps
import pprint
import json
import urllib.parse as parse
from tqdm import tqdm
import numpy as np
import config
from geopy.distance import vincenty

MAPS_KEY = config.MAPS_KEY
gmaps = googlemaps.Client(key=MAPS_KEY)
pp = pprint.PrettyPrinter(indent=4)

In [2]:
df = pd.read_csv('listing_data.csv')
df.head()

,listing_id,date,addr_lat,addr_lon,agent_id
0,1110988,2018-09-12,40.780602,-73.956398,98438.0
1,1272052,2018-09-12,40.758301,-73.959503,12749.0
2,1280705,2018-09-12,40.593800,-73.974503,234944.0
3,1222165,2018-09-12,40.770401,-73.963699,107927.0
4,1238968,2018-09-12,40.709099,-74.013702,212974.0


#### Seems like some agent's ids are missing. Since the exercise states that we need to identify the agent who has travelled the most in one day between two listings, I will be dropping those rows.

In [3]:
df.count()

listing_id    399391
date          399391
addr_lat      399391
addr_lon      399391
agent_id      399274
dtype: int64

#### Very interesting to see that there aren't many unique values. I am assuming because agents can show the same apartment until it is off the market.

In [4]:
df.nunique()

listing_id    17421
date             17
addr_lat       4518
addr_lon       4558
agent_id       8423
dtype: int64

#### Sorting by the dates and agent in order to loop through the rows. Dropped N/As 

In [5]:
sorted_df = df.sort_values(by=['date', 'agent_id']).dropna().reset_index()
sorted_df.head(10)

,index,listing_id,date,addr_lat,addr_lon,agent_id
0,110132,1331217,2018-09-01,40.763302,-73.970100,7335.0
1,110235,1341962,2018-09-01,40.763302,-73.970100,7335.0
2,163089,1350380,2018-09-01,40.763302,-73.970100,7335.0
3,381637,1330931,2018-09-01,40.765598,-73.976898,7335.0
4,62443,1352194,2018-09-01,40.780300,-73.986900,7337.0
5,128146,1305816,2018-09-01,40.769100,-73.981598,7340.0
6,256850,1317380,2018-09-01,40.769100,-73.981598,7340.0
7,288710,1357088,2018-09-01,40.769100,-73.981598,7340.0
8,319500,1283629,2018-09-01,40.769100,-73.981598,7340.0
9,381517,1325545,2018-09-01,40.769100,-73.981598,7340.0


#### Discovered that there were rows with Latitude of 1. Made assumption that latitude 1 (country = Colombia) is not a StreetEasy listing so dropping those rows

In [6]:
sorted_asc = df.sort_values(by=[ 'date', 'agent_id'], ascending=True).dropna().reset_index()
sorted_asc = sorted_asc.drop(sorted_asc[sorted_asc.addr_lat == 1].index)

#### Looping through the sorted rows and collected the first address as the origin and the following row if it matched date and agent made it the destination

In [7]:
origins = []
destinations = []
agent_id = []
date = []

for i in tqdm(range(sorted_asc['listing_id'].count())):
    try:
        if sorted_asc['date'][i] == sorted_asc['date'][i+1] and sorted_asc['agent_id'][i] == sorted_asc['agent_id'][i+1]:
            origins.append(str(sorted_asc['addr_lat'][i]) + "," + str(sorted_asc['addr_lon'][i]))
            destinations.append(str(sorted_asc['addr_lat'][i+1])+ "," + str(sorted_asc['addr_lon'][i+1]))
            agent_id.append(sorted_asc['agent_id'][i])
            date.append(sorted_asc['date'][i])
        else:
            continue
    except KeyError:
        print("No matching agent and date for row: ", i)

 81%|████████  | 321740/399266 [00:26<00:06, 12080.38it/s]

No matching agent and date for row:  319252
No matching agent and date for row:  319253


 82%|████████▏ | 325644/399266 [00:26<00:06, 12079.15it/s]

No matching agent and date for row:  323859
No matching agent and date for row:  323860


 86%|████████▋ | 345274/399266 [00:28<00:04, 12070.13it/s]

...

No matching agent and date for row:  373159
No matching agent and date for row:  373160


 99%|█████████▉| 394706/399266 [00:32<00:00, 12060.69it/s]

No matching agent and date for row:  393201
No matching agent and date for row:  393202


100%|██████████| 399266/399266 [00:33<00:00, 12069.55it/s]

No matching agent and date for row:  397793
No matching agent and date for row:  397794


#### At first I was going to try to loop all the address and make API calls to Google Maps but the performance was not great. Then I thought I could just calculate the distance using geopy functions and came accross the vincenty formula which takes into account Eart being an oblate ellipsoid!

In [8]:
vin_dist = []
for o in tqdm(range(len(origins))):
    vin_dist.append(vincenty(origins[o], destinations[o]).miles)   

100%|██████████| 267161/267161 [00:11<00:00, 22488.71it/s]


#### Returning the data for the two points in which the distance was the greatest

In [9]:
max_dist = np.argmax(vin_dist)
print('index of max value: ', max_dist)
print(origins[max_dist], destinations[max_dist], agent_id[max_dist], date[max_dist])

index of max value:  65535
40.85630035,-73.86599731 40.53900146,-74.21549988 176009.0 2018-09-05


#### Made Google Maps API call to obtain Driving directions for the locations. Assumed that the agent didn't walk from the Bronx to Staten Island

In [13]:
gmap = gmaps.directions(origin = origins[max_dist], destination = destinations[max_dist], mode='driving')
print("On" , date[max_dist] , "Agent:", agent_id[max_dist], "had to travel for ", gmap[0]['legs'][0]['duration']['text'], ". Making it the agent who travelled the longest in one day")

On 2018-09-05 Agent: 176009.0 had to travel for  1 hour 0 mins . Making it the agent who travelled the longest in one day


# Cheers!